In [2]:
import pandas as pd
import os
import zipfile
from datetime import datetime
import re

In [21]:
def extract_files(parquet_main_category, parquet_sub_category, zip_date='tennis_data.zip', start_date='20240201', end_date='20240331'): 
    raw_datas_path = '../data/raw' 
    main_file_path = os.path.join(raw_datas_path, zip_date) 
    zips_saving_path = os.path.join(raw_datas_path, 'zips') 
    parquet_saving_path = os.path.join(raw_datas_path, 'extracted')
    
    start_date = datetime.strptime(start_date, '%Y%m%d')
    end_date = datetime.strptime(end_date, '%Y%m%d')
    
    with zipfile.ZipFile(main_file_path, 'r') as zf: 
        all_files = sorted(zf.namelist()) 
        print(f"Found {len(all_files)} files in main zip")
        zf.extractall(zips_saving_path) 
        
    print(f'Extracting parquets in date range from {start_date} to {end_date} with category' f' --> {parquet_main_category} and subcategory --> {parquet_sub_category}') 
    for zip_date in os.listdir(zips_saving_path): 
        file_date = datetime.strptime(zip_date.split('.')[0], '%Y%m%d')
        
        if start_date <= file_date <= end_date: 
            zip_full_path = os.path.join(zips_saving_path, zip_date) 
            with zipfile.ZipFile(zip_full_path, 'r') as zf:
                files_in_zip = zf.namelist()
                
                for parquet_name in files_in_zip:
                    pattern = rf'.*/raw_{parquet_main_category}_parquet/{parquet_sub_category}_\d+.parquet'
                    if re.fullmatch(pattern, parquet_name): 
                        clean_rel_path = parquet_name.replace(f'../../data/raw/raw_{parquet_main_category}_parquet/', '')
                        dest_path = os.path.join(parquet_saving_path, parquet_main_category, parquet_sub_category, clean_rel_path)
        
                        os.makedirs(os.path.dirname(dest_path), exist_ok=True)
        
                        print(f'Extracting {clean_rel_path}')
                        with zf.open(parquet_name) as src, open(dest_path, 'wb') as dst:
                            dst.write(src.read())

In [22]:
extract_files(parquet_main_category='match', parquet_sub_category='event')

Found 60 files in main zip
Extracting parquets in date range from 2024-02-01 00:00:00 to 2024-03-31 00:00:00 with category --> match and subcategory --> event
Extracting event_11974053.parquet
Extracting event_11974066.parquet
Extracting event_11998445.parquet
Extracting event_11998446.parquet
Extracting event_11998447.parquet
Extracting event_11998448.parquet
Extracting event_11998449.parquet
Extracting event_11998450.parquet
Extracting event_11998451.parquet
Extracting event_11998456.parquet
Extracting event_11998459.parquet
Extracting event_11998666.parquet
Extracting event_11998667.parquet
Extracting event_11998670.parquet
Extracting event_11998671.parquet
Extracting event_11998672.parquet
Extracting event_11998674.parquet
Extracting event_11998675.parquet
Extracting event_11998676.parquet
Extracting event_11998772.parquet
Extracting event_11998774.parquet
Extracting event_11998780.parquet
Extracting event_11998782.parquet
Extracting event_11998783.parquet
Extracting event_11998784

In [17]:
def create_parquet_files(parquet_main_category, parquet_sub_category):
    datas_path = f'../data/raw/extracted/{parquet_main_category}/{parquet_sub_category}'
    save_dir = '../data/raw/parquets'
    save_path = os.path.join(save_dir, f'{parquet_main_category}_{parquet_sub_category}.parquet')

    os.makedirs(save_dir, exist_ok=True)

    all_files = os.listdir(datas_path)

    dfs = []
    for parquet_name in all_files:
        parquet_path = os.path.join(datas_path, parquet_name)
        df = pd.read_parquet(parquet_path, engine='pyarrow')
        dfs.append(df)

    result = pd.concat(dfs, ignore_index=True)
    print("Saving to:", save_path)
    result.to_parquet(path=save_path, index=False)

    return result

In [23]:
create_parquet_files('match', 'event')

Saving to: ../data/raw/parquets\match_event.parquet


,match_id,first_to_serve,home_team_seed,away_team_seed,custom_id,winner_code,default_period_count,start_datetime,match_slug,final_result_only
0,11974049,None,None,None,RNfskjg,2,3,1707051600,belgium-croatia,False
1,11974052,None,None,None,DQfsujp,2,3,1706882400,serbia-slovakia,False
2,11974053,None,None,None,NNfsQTf,1,3,1706878800,switzerland-netherlands,False
3,11974065,None,None,None,PNfsFFg,1,3,1707055200,israel-czech-republic,False
4,11974066,None,None,None,hTfsrpg,2,3,1706871600,ukraine-usa,False
...,...,...,...,...,...,...,...,...,...,...
16868,12213482,2,None,None,TpwcsODcd,1,3,1711978500,benito-fernandez-baldovinetti,False
16869,12213483,1,None,None,QxJcsjbVc,1,3,1711978200,schinteie-kypriotis,False
16870,12213484,2,None,None,yLPbsvyXb,1,3,1711978200,hornitschek-hornung,False
16871,12213486,2,None,None,qzqbstjac,1,3,1711978200,agostinetto-cano,False


In [24]:
df = pd.read_parquet('../data/raw/parquets/match_event.parquet', engine='pyarrow')
df.shape

(16873, 10)